In [28]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


## Getting the list of files
from pathlib import Path

fish_dir = Path('./fishes/Fish_Dataset/Fish_Dataset')
file_path = list(fish_dir.glob(r"**/*.png"))

file_path[:10]

[WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00001.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00002.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00003.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00004.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00005.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00006.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00007.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00008.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00009.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00010.png')]

In [29]:
## Getting the labels of the images
from os import path
import pandas as pd

labels = list(map(lambda f: path.split(path.split(f)[0])[1], file_path))

labels = pd.Series(labels)
labels.unique()

array(['Black Sea Sprat', 'Black Sea Sprat GT', 'Gilt-Head Bream',
       'Gilt-Head Bream GT', 'Hourse Mackerel', 'Hourse Mackerel GT',
       'Red Mullet', 'Red Mullet GT', 'Red Sea Bream', 'Red Sea Bream GT',
       'Sea Bass', 'Sea Bass GT', 'Shrimp', 'Shrimp GT',
       'Striped Red Mullet', 'Striped Red Mullet GT', 'Trout', 'Trout GT'],
      dtype=object)

In [30]:
## Images series
images = pd.Series(file_path).astype(str)
images.head()

0    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
1    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
2    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
3    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
4    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
dtype: object

In [31]:
## Creating dataframe of fishes with their labels
fishes = pd.concat([images, labels], axis=1)
fishes.columns = ["image", "label"]
fishes = fishes[fishes["label"].apply(lambda x: x[-2:] != "GT")].sample(frac=1).reset_index(drop=True)
fishes.shape

(9000, 2)

In [32]:
## Splitting data
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(fishes, test_size=0.15, random_state=15, shuffle=True)

print(train_data.shape)
print(test_data.shape)

(7650, 2)
(1350, 2)


In [33]:
print(train_data.label.value_counts())
print("###")
print(test_data.label.value_counts())

Gilt-Head Bream       860
Red Sea Bream         859
Shrimp                857
Red Mullet            857
Sea Bass              851
Black Sea Sprat       847
Striped Red Mullet    847
Trout                 838
Hourse Mackerel       834
Name: label, dtype: int64
###
Hourse Mackerel       166
Trout                 162
Black Sea Sprat       153
Striped Red Mullet    153
Sea Bass              149
Shrimp                143
Red Mullet            143
Red Sea Bream         141
Gilt-Head Bream       140
Name: label, dtype: int64


In [34]:
## Image Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
input_shape = (224, 224, 3)

train_gen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2, zoom_range=0.2)
test_gen = ImageDataGenerator()

train_images = train_gen.flow_from_dataframe(dataframe=train_data,
                                             x_col="image",
                                             y_col="label",
                                             target_size=img_size,
                                             class_mode="categorical",
                                             batch_size=32,
                                             subset="training",
                                             seed=15,
                                             color_mode="rgb",
                                             shuffle=True)

validation_images = train_gen.flow_from_dataframe(dataframe=train_data,
                                                  x_col="image",
                                                  y_col="label",
                                                  target_size=img_size,
                                                  class_mode="categorical",
                                                  batch_size=32,
                                                  subset="validation",
                                                  seed=15,
                                                  color_mode="rgb",
                                                  shuffle=True)

test_images = test_gen.flow_from_dataframe(dataframe=test_data,
                                           x_col="image",
                                           y_col="label",
                                           target_size=img_size,
                                           class_mode="categorical",
                                           batch_size=32,
                                           color_mode="rgb",
                                           shuffle=False)

Found 6120 validated image filenames belonging to 9 classes.
Found 1530 validated image filenames belonging to 9 classes.
Found 1350 validated image filenames belonging to 9 classes.


In [35]:
from keras import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten, Dense, Conv2D, Activation

model = Sequential([Conv2D(32, (3, 3), activation='relu', strides=2, input_shape=input_shape, padding="same"),
                    MaxPooling2D(pool_size=(2, 2)),
                    Conv2D(32, (3, 3)), Activation('relu'), MaxPooling2D(pool_size=(2, 2)),
                    Flatten(), Dense(128, activation='relu'),  Dropout(0.15),
                    Dense(128, activation='relu'), Dropout(0.2), Dense(9, activation="softmax")])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 112, 112, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 54, 54, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 54, 54, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 23328)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [36]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [37]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger

rate_reduction = ReduceLROnPlateau(monitor="val_accuracy", patience=5, verbose=1, factor=0.2, min_lr=0.0001)
early_stop = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)
csv_logger = CSVLogger("model_history_log.csv", append=True)

In [38]:
history = model.fit(train_images, validation_data=validation_images, epochs=50, verbose=1,
                    callbacks=[rate_reduction, early_stop, csv_logger])

Epoch 1/50
192/192 [==============================] - 109s 568ms/step - loss: 1.6207 - accuracy: 0.4046 - val_loss: 0.9271 - val_accuracy: 0.7092
Epoch 2/50
192/192 [==============================] - 108s 563ms/step - loss: 0.8358 - accuracy: 0.7016 - val_loss: 0.5924 - val_accuracy: 0.7765
Epoch 3/50
192/192 [==============================] - 109s 566ms/step - loss: 0.5534 - accuracy: 0.8039 - val_loss: 0.4506 - val_accuracy: 0.8418
Epoch 4/50
192/192 [==============================] - 109s 566ms/step - loss: 0.4273 - accuracy: 0.8515 - val_loss: 0.3525 - val_accuracy: 0.8771
Epoch 5/50
192/192 [==============================] - 108s 565ms/step - loss: 0.3094 - accuracy: 0.8905 - val_loss: 0.2250 - val_accuracy: 0.9255
Epoch 6/50
192/192 [==============================] - 109s 566ms/step - loss: 0.2220 - accuracy: 0.9222 - val_loss: 0.1897 - val_accuracy: 0.9314
Epoch 7/50
192/192 [==============================] - 108s 565ms/step - loss: 0.1989 - accuracy: 0.9324 - val_loss: 0.2751 -

In [39]:
import numpy as np

prediction = np.argmax(model.predict(test_images), axis=1)
predicted_labels = test_images.class_indices
predicted_labels = dict((v, k) for k, v in predicted_labels.items())
predictions = [predicted_labels[k] for k in prediction]

In [40]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

y_test = test_data.label
print(classification_report(y_test, predictions))

print(accuracy_score(y_test, predictions))
confusion_matrix(y_test, predictions, normalize='true')

                    precision    recall  f1-score   support

   Black Sea Sprat       0.86      0.97      0.91       153
   Gilt-Head Bream       0.97      0.78      0.87       140
   Hourse Mackerel       0.70      0.99      0.82       166
        Red Mullet       0.93      0.99      0.96       143
     Red Sea Bream       0.93      0.94      0.94       141
          Sea Bass       0.94      0.73      0.82       149
            Shrimp       0.78      0.99      0.87       143
Striped Red Mullet       0.96      0.65      0.77       153
             Trout       0.96      0.78      0.86       162

          accuracy                           0.87      1350
         macro avg       0.89      0.87      0.87      1350
      weighted avg       0.89      0.87      0.87      1350

0.8688888888888889


array([[0.96732026, 0.        , 0.02614379, 0.        , 0.        ,
        0.00653595, 0.        , 0.        , 0.        ],
       [0.        , 0.77857143, 0.1       , 0.        , 0.05      ,
        0.04285714, 0.        , 0.00714286, 0.02142857],
       [0.        , 0.        , 0.9939759 , 0.        , 0.0060241 ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.99300699, 0.        ,
        0.        , 0.00699301, 0.        , 0.        ],
       [0.        , 0.        , 0.04255319, 0.        , 0.94326241,
        0.        , 0.0070922 , 0.0070922 , 0.        ],
       [0.10067114, 0.01342282, 0.08053691, 0.        , 0.        ,
        0.73154362, 0.04697987, 0.01342282, 0.01342282],
       [0.        , 0.        , 0.        , 0.00699301, 0.        ,
        0.        , 0.99300699, 0.        , 0.        ],
       [0.05228758, 0.00653595, 0.01960784, 0.06535948, 0.        ,
        0.        , 0.20915033, 0.64705882, 0.        ],


In [41]:
from tensorflowjs import converters

converters.save_keras_model(model, "tf-js-model")

print("Saved successfully")

Saved successfully


In [42]:
print(test_images.filenames[:6])
print(predictions[:6])

['fishes\\Fish_Dataset\\Fish_Dataset\\Red Mullet\\Red Mullet\\00785.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Red Mullet\\Red Mullet\\00419.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Trout\\Trout\\00695.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Sea Bass\\Sea Bass\\00540.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00862.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Gilt-Head Bream\\Gilt-Head Bream\\00176.png']
['Red Mullet', 'Red Mullet', 'Trout', 'Trout', 'Black Sea Sprat', 'Gilt-Head Bream']
